# 네이버_여행지정보_크롤러

In [1]:
# 네이버에서 검색어 입력받아 검색 한 후 블로그 메뉴를 선택하고
# 오른쪽에 있는 검색옵션 버튼을 눌러서
# 정렬 방식과 기간을 입력하기

#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys
import os
import pandas as pd
import numpy as np

from selenium import webdriver
import time

# tqdm : for문 돌릴 때 진행상황
from tqdm import tqdm_notebook

# warning 메시지 무시하는 코드
import warnings ; warnings.filterwarnings(action='ignore')

In [2]:
spot = input('1.크롤링할 여행지는 어디입니까?: ')

1.크롤링할 여행지는 어디입니까?: 경복궁


In [3]:
#Step 1. 크롬 웹브라우저 실행
path = "chromedriver.exe"  # 맥 기준  # 윈도우는 "chromedriver.exe"

driver = webdriver.Chrome(path)
# 사이트 주소는 네이버
driver.get('http://www.naver.com')
time.sleep(2)

#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_id("query")
element.send_keys(spot)  # query_txt는 위에서 입력한 키워드
element.submit()
time.sleep(1)

## 위치, 방문자 리뷰 수, 블로그 리뷰 수 수집하기

현재 네이버에 "경복궁"이 검색된 상태(네이버 지도에 들어가지 않음)

In [4]:
# 위치
location_css = """//*[@id="place_main_ct"]/div/div/div/div[2]/div[3]/div/div[2]/div/ul/li[1]/span"""
location = driver.find_element_by_xpath(location_css).text

# 방문자 리뷰 수
passenger_css = """//*[@id="place_main_ct"]/div/div/div/div[2]/div[1]/div/div/a[1]/span[2]"""
passenger_total = driver.find_element_by_xpath(passenger_css).text
passenger_total = int(passenger_total[7:].replace(',', ''))

# 블로그 리뷰 수
blog_css = """//*[@id="place_main_ct"]/div/div/div/div[2]/div[1]/div/div/a[2]/span"""
blog_total = driver.find_element_by_xpath(blog_css).text
blog_total = int(blog_total[7:].replace(',', ''))

driver.close()
print(f"여행지 '{spot}'의 위치는 {location}입니다.")
print(f"{spot}의 네이버 총 방문자 리뷰 수는 {passenger_total}건 이고, 총 블로그 리뷰 수는 {blog_total}건 입니다.")

여행지 '경복궁'의 위치는 서울 종로구 사직로 161 경복궁입니다.
경복궁의 네이버 총 방문자 리뷰 수는 6189건 이고, 총 블로그 리뷰 수는 9643건 입니다.


## 네이버 지도 속 방문자 리뷰 크롤링
**네이버 마이 플레이스의 메인 검색창** -> **spot** 검색 -> **'방문자 리뷰 더보기'** 클릭 -> 리뷰 수집
  
- 해당 장소의 총 평점 : 전체 평균 평점

- userID 별 평점 수집 :  
   1) 일반(텍스트 포함)리뷰 : `리뷰 수`,`userID`, `user의 평균 평점`, `user의 해당 명소 평점`, `텍스트 후기`  
   2) 별점(텍스트 없는)리뷰 : `리뷰 수`,`userID`, `user의 평균 평점`, `user의 해당 명소 평점`, `텍스트 후기`


In [56]:
spot

'경복궁'

In [42]:
#Step 1. 크롬 웹브라우저 실행
path = "chromedriver.exe"  # 맥 기준  # 윈도우는 "chromedriver.exe"
driver = webdriver.Chrome(path)

# 사이트 주소는 네이버 마이 플레이스
driver.get('https://m.naver.com/')
time.sleep(2)

#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_xpath("""//*[@id="MM_SEARCH_FAKE"]""").click()
element_search = driver.find_element_by_xpath("""//*[@id="query"]""")
element_search.send_keys(spot)  # query_txt는 위에서 입력한 키워드
element_search.submit()
time.sleep(1)

#Step 3. '방문자 리뷰 더보기' 클릭
driver.find_element_by_xpath("""//*[@id="place-main-section-root"]/div/div[3]/div[2]/a""").click()

### 해당 장소의 총 평점  : 전체 평균 평점

In [148]:
# 전체 평균 평점
total_avg_score_css = """//*[@id="app-root"]/div/div[2]/div[4]/div[4]/div[2]/div/div/div/div[1]/div[2]"""
total_avg_score = driver.find_element_by_xpath(total_avg_score_css).text
total_avg_score = float(total_avg_score[:-2])

### userID 별 평점 수집 

In [80]:
# 스크롤 다운 
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 99999999)")
    time.sleep(1)
scroll_down(driver) # 스크롤 다운

# 일반(텍스트 포함)리뷰 수
normal_css = """//*[@id="app-root"]/div/div[2]/div[5]/div[4]/div[4]/h2/span"""
normal = driver.find_element_by_xpath(normal_css).text
normal = int(normal.replace(',', ''))

# 별점(텍스트 없는)리뷰 수
star_css = """//*[@id="app-root"]/div/div[2]/div[5]/div[4]/div[5]/h2/em"""
star = driver.find_element_by_xpath(star_css).text
star = int(star.replace(',', ''))

driver.close()

print("전체", passenger_total, '개의 리뷰 중')
print("일반(텍스트 포함)리뷰는 총", normal, "개")
print("별점(텍스트 없는)리뷰는 총", star, "개")

전체 6189 개의 리뷰 중
일반(텍스트 포함)리뷰는 총 5335 개
별점(텍스트 없는)리뷰는 총 855 개


#### 일반(텍스트 포함)리뷰 : userID, user의 평균 평점, user의 해당 명소 평점, 텍스트 후기

In [47]:
#Step 1. 크롬 웹브라우저 실행
path = "chromedriver.exe"  # 맥 기준  # 윈도우는 "chromedriver.exe"
driver = webdriver.Chrome(path)

# 사이트 주소는 네이버 마이 플레이스
driver.get('https://m.naver.com/')
time.sleep(2)

#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_xpath("""//*[@id="MM_SEARCH_FAKE"]""").click()
element_search = driver.find_element_by_xpath("""//*[@id="query"]""")
element_search.send_keys(spot)  # query_txt는 위에서 입력한 키워드
element_search.submit()
time.sleep(1)

#Step 3. '방문자 리뷰 더보기' 클릭
driver.find_element_by_xpath("""//*[@id="place-main-section-root"]/div/div[3]/div[2]/a""").click()

In [48]:
# 더보기를 누를 횟수
push1 = normal//10
print('일반(텍스트 포함)리뷰 페이지 수 :', push1)

for i in tqdm_notebook(range (push1)):
    scroll_down(driver) # 스크롤 다운
    # [더보기] 클릭
    driver.find_element_by_css_selector("div._2kAri").click()

In [50]:
review_overalls = driver.find_elements_by_css_selector('._2Cv-r')
len(review_overalls)

5335

In [53]:
total_normal = []
user_id = []
user_avg = []
user_Spot_avg =[]
user_text = []

for i in tqdm_notebook(range (normal)):
    review_overalls = driver.find_elements_by_css_selector('._2Cv-r')
    information = review_overalls[i].text.split('\n')
    total_normal.append(information)
    
    if '원문보기' in information:
        information = information[:-1]

    # user_id
    user_name = information[0]
    user_id.append(user_name)

    # user_avg
    user_avg_value = float(information[3])
    user_avg.append(user_avg_value)

    # user_Spot_avg
    if information[-3] == '리뷰이미지':
        user_Spot_avg_value = float(information[-2])
    else: user_Spot_avg_value = float(information[-3])
    user_Spot_avg.append(user_Spot_avg_value)

    # user_text
    text = information[-1]
    if '영수증' in text:
        user_text.append('텍스트 후기 없음')
    else: user_text.append(text)
    
    print(i+1, user_name, user_avg_value, user_Spot_avg_value, text )

print(user_id)
print(user_avg)
print(user_Spot_avg)
print(user_text)

1 라떼 4.9 5.0 좋네요~~
2 히카루 4.9 5.0 2021.01.171번째 방문영수증
3 장다르메 4.5 5.0 2021.01.161번째 방문영수증
4 kj9948 5.0 5.0 감사합니다
5 starry night 4.9 5.0 엄청넓어요
6 coa**** 4.7 4.0 조아요
7 kms00680 4.1 5.0 겨울 경복궁의 경회루
8 Swan 4.1 4.0 또 방문하겠습니다
9 김토리 4.8 4.5 늘가도 좋은 곳
10 chimmini 4.7 5.0 산책하기 정말 좋아요
11 뿌다 4.8 5.0 #대한민국경복궁관람
12 NN 5.0 5.0 굿
13 로빈 4.2 5.0 걷기 좋아요
14 낌낌낌 4.9 5.0 굿
15 판타스틱하게 4.3 4.5 굿
16 태산 4.6 5.0 굿
17 Serena 4.5 5.0 좋았어요 ㅎㅎ 경회루 못들어가서 아쉬워요!
18 강철소나무 4.3 5.0 진짜 최고...👍 내년에 꼭 다시가려구요
19 해피 4.9 5.0 최고였어여!!!!
20 dldPwls 4.9 4.5 넘추웟음 ㅜ
21 Yein kim 4.8 5.0 이쁜데 밤이라 사진이 잘나오는편은 아니였습니다 그리고 한파때 가서 너무너무 추운거 빼고는 완벽했습니다 날씨 안추울때 또 가고싶어요.
22 유깅 5.0 5.0 야경 너무 예쁘고 재밌었어요!!!
23 도현 5.0 5.0 마지막 관람인데 좀 추웠지만 좋았어요~
24 ian**** 4.9 4.0 관람 마지막날 갔는데 너무 춥더라구요~사진만 몇장 후딱 찍었네요~한겨울 관람은 비추~그래도 조명은 잘 설치해놓아서 예뻤습니다~~
25 냥곰냥곰 5.0 5.0 너무 추운날이였지만 정말 너무 예뻐서 추운것도 잊고 이곳저곳 돌아다녔네요~^^
26 muse102 4.3 4.0 추운데 좋았어요
27 예랑 4.9 5.0 추운 날이었지만 마지막 날, 마지막 손님으로 좋은 추억이 되었습니다.
28 Rexford Chung 5.0 5.0 너무 아름답고 감성있는 시간이었습니다.
29 yjyjjwrhri5 5.0 5.0 아이들이 생각보다 더 신나하

147 곽동엽 5.0 5.0 잘보다가 갑니다
148 hye**** 4.5 5.0 정말 예쁘고 정말 추워요 패딩 필수
149 최하영 5.0 5.0 바람이 불어서 추웠지만. 야경이 이쁘고 좋았어요
150 핑크 쏭 5.0 5.0 날씨는 좀 추웠지만 너무 예쁘고 좋았습니다. 여기 일하시는 분들 어쩜 다 이렇게 친절하신지요! 모두 고생많으십니다~ 좋은 추억 만들고 가요~^^펼쳐보기
151 Peter 4.9 5.0 밤에 보는 경복궁은 정말 아름답고 몽환적입니다. 다만 오늘은 너무 추워서 손이 아프네요, 장갑 꼭 끼세요 ㅎㅎㅎ
152 RVRVRVE 5.0 5.0 완전 좋은데 불빛이 더 많았으면 좋겠네요 깜깜한데는 너무 시껌쓰
153 ujs 4.8 5.0 좋아요
154 ssun구름 4.8 5.0 감동적이었습니다
155 sonxab6ix 4.7 5.0 최고♡ 야경이 너무너무 아름다워욥
156 지나또별 5.0 5.0 좋았습니다!
157 bab**** 4.5 5.0 밤에보는 야경이 너무 멋졌어요. 코로나로 인해 후딱보고 온게 아쉬웠어요.
158 보금 4.6 5.0 7-8년전에 보고 또갓는데 여전히 이뻐요 코로나라 걱정햇은데 관리요원분들이 계속 관리해주시고 거리두기 표시도 되어잇어서 실내보다 낫네요 ㅎ
159 당콩 4.1 1.5 볼꺼 1도 없더라..
160 tls**** 5.0 5.0 정말 너무너무 이쁘고 멋있었어요
161 꽃잎바람 4.5 4.0 춥긴했지만 분위기좋았어요
162 크으 5.0 5.0 너무 이쁘고 좋은 추억 만들었습니다.
163 김수빈 4.7 5.0 돈이 안아까워요 넘 좋아요
164 튜됴미 4.9 4.5 너무 좋았습니당
165 shalala507 4.0 4.0 추웠지만 그래도 좋았어요
166 big**** 5.0 5.0 고요하고 아름답네요
167 빵좋아 5.0 5.0 정말 최고
168 졸부 4.1 5.0 너무너무 좋아요!! ♥
169 상콤이 4.9 4.5 야간으로 보니깐 굉장히 멋있었습니다
170 상윤이는귀여워 5.0 5.0 쩐다쩔어ㅜㅜ
171 jan**** 4.7 5

311 이나로리 5.0 5.0 역시 경복궁 야경 너무 아름다웠고, 근무하시는 분들너무 친절하셨고 좋았어요.
312 kim**** 5.0 5.0 춥긴했지만 좋았어요 생각보다 넓네요
313 jsh**** 4.8 4.0 생각했던거보단 좀 어두웠어요! 그래도 곳곳에 경비,관리해주시는 분들이 안내해주셔서 잘 관람하였습니다.
314 gks**** 5.0 5.0 야간개장이 너무좋았습니다 경회루 호수에 나뭇잎이 없었으면 더 좋았을것같아요
315 let**** 4.6 5.0 바람이 불지 않아 그렇게 춥지 않았어요.
316 aegyojin 5.0 5.0 평일이어서 사람도 완전 없고 엄청 춥지도 않고... 딱 좋은 날씨에 힐링하다 갑니다!!
317 푸들치킨 4.8 5.0 코로나 2.5단계 격상하면서 어떻게든 바깥세상 좀 보겠다고 티켓여유분이 많길래 예매하고 갔는데 사람이 바글바글했으면 사진찍기도 궁을 구경하기에도 힘들었을텐데 생각보다 없어서 더 좋았고 물에비친 궁전들 너무 이쁘게 보다 갑니다
318 세컨컵 4.6 5.0 안내자분들 너무 친절하셔서 기분좋게 다녔어요. 유모차 보시고는 아기 춥겠다며 본인 핫팩도 나눠주시고 초코파이도 주시고. 사진도 정말 친절히 찍 어주시고. 너무너무 감사하단말 드리고싶어요펼쳐보기
319 lkn**** 5.0 5.0 안내해 주시는 분들이 친절하셨습니다.
320 아만다 4.5 4.5 매번 매진이라 드디어.갔다왔는데 너무 좋았어요 :) 춥지도 않고, 궁 안내부에 안내해주시는 분들이 많아서 잘 보고 왔어요
321 frn 5.0 5.0 안내하시는 분들이 너무 친절하셔서 혼자 갔는데도 즐겁게 잘 다녀왔습니다!ㅎㅎ 다음에는 낮에도 꼭 방문하고싶어요~!
322 y2h**** 5.0 5.0 밤에 보는 경복궁은 달랐습니다. 시간내서 한번쯤 밤에 볼만한 풍경이었습니다. 아이들도 예쁘다며 좋아했습니다. 펼쳐보기
323 루시아 4.4 4.0 다 좋은데, 생각보다 많이 어두워서..동절기 땐 6시 입장 시작이라든지 조금은 밝게 관람해도 좋을 것 같아요.
324 ouioui 5

440 하잉 5.0 5.0 좋습니다~
441 아용용 5.0 5.0 낮이랑 또다른 느낌이였어요. 너무 아름답고 감동받고왔습니다.
442 lkji9674 4.2 5.0 너무 이쁘고 좋았습니다
443 준준 5.0 5.0 모든게 완벽했어요
444 좋아좋아 4.8 5.0 정말 멋져요~~~
445 무적LG 5.0 5.0 아들이랑 같이같는데 날씨도 안춥고 구경잘했습니다
446 wun**** 3.8 5.0 이뻐요!!
447 ccw**** 5.0 5.0 진짜 예뻤어요 낮에도 가보고싶어요!
448 봉그미 3.3 5.0 너무 아름다운 고궁의 밤이었습니다. 예약 안내에는 저녁 9:30분까지라고 뜨지만 실제로는 일부 직원분들 퇴근시간과 입장마감이 오후 8:30 쯤이니 참고해서 빨리 입장 하시는 게 좋을 것 같아요.
449 kyz**** 4.7 5.0 야간에 보니 경치가 더 예쁘네요~
450 채민석 4.8 5.0 날씨가 별로 안추워서 한복입구 갔었는데 정말 맘에들었어요👍 사진도 엄청 찍고 색감이 진짜👍 구경하는 내내 행복했습니다~
451 mnms7 4.5 5.0 정말 만족해요 아름답고 행복한 추억 만들었네요
452 UKIC 5.0 5.0 너무너무 예쁘고 좋았습니다 야간 개장은 처음이었는데 느낌이 다르더군요! 안내해주시는분들 정말 추우셨을텐데 친절하게 하나하나 알려주시고 인사해주시는데 정말 감사했습니다
453 호텔리어라 4.5 5.0 춥긴 했지만, 밤에 아름다운 한국의 궁을 볼 수 있어서 좋았습니다. 추운 날 방문객들의 안전을 위해 곳곳에 직원들이 배치돼어 있어 길을 찾기도 안내 받기도 편했습니다.
454 안녕난몽키야 5.0 5.0 좋은 추억이 되었습니다
455 kjs**** 5.0 5.0 너무 이쁘고 좋았습니다. 안내원분들도 너무 친절해요
456 dew 5.0 5.0 코로나가 한창이라 고민끝에 갔는데 너무 잘다녀왔다고 생각했어요 야경이어야만 보고 느낄수 있는 경복궁의 모습에 많이 감탄하고 왔습니다!!
457 koo**** 2.8 4.5 좋습니다 추웠지만 괜찮아요
458 신신 4.1 5

578 kookookoo3 4.5 5.0 내년에 또갈거에요!!!
579 태준군 5.0 5.0 날씨가 조금 쌀쌀했지만 딸이랑 소중한 추억 만들고 왔습니다^^
580 ebb**** 4.6 4.5 너무이뻤어요! 단지... 다들 야경모드 카메라를 챙겨가세요 이쁨을 담지못하여 아깝습니다!🖤
581 상수의보온병 4.5 5.0 꼭 가세요 여러분 두번가세요
582 호이 5.0 5.0 춥지만 이쁘게 잘 보고왔습니다:)
583 순대 4.0 5.0 너무 예쁘고 좋았어요
584 미대13 4.9 5.0 야경 진짜 멋있어요
585 박테리아 4.5 5.0 날씨가 추운날이라 별로 못 있었지만 너무 예뻤어요
586 fhq**** 5.0 5.0 너무추워요 하지만 그 운치는 최고입니다
587 Jijiwon 5.0 5.0 낮과 밤의 경복궁은 굉장한 것 같아요 개인적으로는 밤에 본 경복궁은 정말 멋있었어요^^
588 은별 5.0 5.0 너무너무 예쁘고 관리자분들도 친절하셨어요! 다만 겨울에 가실거면 손핫팩 필수입니다ㅠㅠ
589 jwjw0715 4.8 5.0 밤에 보는 경복궁이 너무 예뻤습니다 근데 엄청 추우니 롱패딩 입으세요
590 이선재 5.0 5.0 너무 이뻐요 진짜!!! 날씨 따듯한날 또 오고싶네요☺️
591 Fine 4.7 5.0 예쁘고 사람별로없어 좋았습니다! 참고로 안에있는 기념품샵에서 커피사먹지마세여 그냥 맥심커피타줍니다ㅋㅋㅋㅋㅋㅋ
592 Hyacintha 5.0 5.0 어두운 시간에 만난 궁은 또 다른 아름다움이 있었어요~ 정말 너무 예쁘다~만 수십번 외치며 왔던 것 같습니다. 좋은 추억을 남길 수 있어서 좋았어요~☺️
593 cha**** 5.0 5.0 퇴근후에 큰 부담없이 고궁의 화려함과 함께 편안한 산책도 가능해 너무 좋았습니다. 낮에 보던 궁도 아름답지만 근정전 경회루등 야경은 정말 기억에 오래 남을 듯 합니다.
594 채널 4.7 5.0 경복궁야경 역시 최고에요
595 소담소담 4.8 4.5 고즈너하고 운치또한 좋습니다
596 도르마무 4.4 5.0 안내해주시는 어르신들이 

...

4623 서녕 3.8 4.5 너무 좋았어요. 해설사의 해설을 못들은게 아쉬워요.코로나 없어지면 해설 들으러 다시가고싶어요.
4624 jjk**** 5.0 5.0 너무좋아요~
4625 작은 자 4.8 4.0 연인들끼리 좋은 테이트 장소인듯 해요
4626 미 미 4.5 4.5 아주 좋았어요~~
4627 옹달 5.0 5.0 달까지 너무 예뻤어요
4628 박군 4.5 5.0 야간관람은 처음이였는데 경회루 주변에 푸른색 조명이 안 어울리는것 외에는 좋았어요~
4629 싸미 4.6 5.0 정말 멋졌어요~!!! 굿굿
4630 꿀별 5.0 5.0 경복궁 야간관람은 처음이었는데 낮에 보는것도 좋지만 밤에 보는 경복궁은 정말 정말 멋있었어요~^^ 특히 경회루와 근정전이 넘 넘 아름다웠고 밝은 보름달까지 함께 비춰주니까 정말 분위기 있는 한편의 작품을 보는듯 했네요~^^ 사실 코로나때문에 많이 고민하다가 방문했는데 공간이 넓고 관람 인원 제한이 있어서 어려움 없었고요, 매표소부터 궁 안의 모든 직원분들이 다 친절하게 대해 주셔서 넘 감사했어요~ 특히 교태전 방향 직원분...펼쳐보기
4631 roo**** 4.5 5.0 날씨가 좋아서 걷기 너무 좋았어요. 밤이라 너무 멋졌어요. 오픈하자마자 가면 사람 많으니 8시쯤 들어가면 한적하게 볼수 있을거 같아요.
4632 sh0445 3.5 5.0 추석에 가족들과 갔는데 너무 좋았어요 다만 어두워서 유모차를 끌고 다니거나 24개월 아기가 막 뛰어다니기는 약간 위험했어요
4633 힐델리다 4.7 5.0 낮에가는 궁과 다른 분위기를 느낄 수 있는 야간개장이예요. 생각보다 많은 분들이 구경 나왔더라구요.
4634 김태윤 5.0 5.0 날씨도 좋고 보릉달이어서 좋았어요~!^^
4635 지선 5.0 5.0 가족,친구,연인 누구와 와도 좋은 것 같아요
4636 안녕 5.0 5.0 너무 좋았어요! 사람들이 많았지만 경복궁이 넓어서 거리두기가 잘 실천되면서 잘 구경할 수 있었어요. 경복궁 너무 아름다웠고 값진 경험이었어요. 또 가고 싶어요!
4637 콩이

4755 지수 5.0 5.0 언제나좋음
4756 sun**** 4.8 4.5 야경이 넘 멋지네요. 낮과는 또 다른 분위기
4757 어붕 4.8 5.0 너어무 좋았습니다. 일전엔 사람이 너무 많아서 바닥에 있는 모래 먼지가 부옇게 떠다닐 정도였는데. 올해는 확실히 사람이 좀 적어서 운치 있게 보고 왔습니다. 좋았어요ㅠㅠ
4758 성북구주민 5.0 5.0 예쁘고 관함하기 좋은 날씨였습니다
4759 yan**** 4.4 5.0 좋았습니다 어두워서 사진은 찍기 힘들어요 눈으로 담으세요 ~
4760 jem 3.4 2.0 관람규역 제한되어있어서ㅜ아쉬워요
4761 jem 3.4 2.0 관람 구역이 제한되어져 있는데다가 촬영한다고 제한되어서 지장이 ㅇㅆ었습니다
4762 Junip 5.0 5.0 분위기 너무 좋고 사람도 적어서 완전 강추
4763 웅 5.0 5.0 너무 아름다웠습니다.
4764 rla**** 3.5 3.5 야간개장 충분히 이뻤고, 공연 촬영하는 날에 가서 공연을 즐길 수 있어 좋았습니다. 다만, 야간개장이라 그런지 구경할 수 있는 공간이 한정적이어서 아쉽습니다.
4765 bab**** 4.3 5.0 경회루에서 영상 촬영이 있어 조금 소란했지만 고궁의 밤 풍경은 정말 아름다웠다
4766 정리 4.3 4.0 평일이라 사람 많이 없어서 좋았어요 운치있고 도심 속 산책할 수 있어 행복했어요!
4767 시나몬카페라떼 5.0 5.0 야간개장 너무너무 감사합니다. 어려운시기에 궁 보면서 마음이 편안해졌던 것 같아요.
4768 무 3.4 3.5 낮이 아닌 조명이 켜진 가을 밤에 보는 경회루가 참 좋았어요. 날을 잘 선택해서인지 관람객도 많지는 않았고요. 아쉬운 점을 적어 보자면, 첫째, 홍보 물 촬영날이었는지 촬영 스탭들과 퍼포먼스 댄서들이 촬영하고 있는데 음악소리가 크고 조명물들이 설치되어 있어 온전히 궁만을 촬영하고 싶었는데 그 점이 아쉬웠구요( 혹시 그 홍보물이 한국을 소개하는 홍보영상이라면, 의상과 음악은 완전 형편없다는 생각이 듭니다. 갓만 썻다 뿐이지, 뭔 ...

4893 삥꾸뿌꾸 4.9 5.0 너무 좋았어요~ 아이가 너무 좋아해요
4894 회색물고기 5.0 5.0 너무 분위기 좋고 사진 찍기도 좋고 너무 좋았습니다!
4895 나대리 4.2 5.0 너무 좋았어요 한번 더 다녀올 생각입니다 발권표 티켓줄이 길었지만 금방 줄더라구요 ~~!!펼쳐보기
4896 ehdrb231 5.0 5.0 고궁의 특별함을 느낄 수 있어 최고였습니다
4897 lyl**** 4.6 4.0 입장권을 QR로 바꿔주세요 티켓팅할때 줄도 엄청길어서 거리두기도 안되고 ..코로나때메 넘 위험한거같아요 종이낭비기도 한것 같고펼쳐보기
4898 JHH 3.6 4.0 더운여름이 지나고 가을 저녁에 구경을 하니 선선하게 밤바람을 느낄수 있어서 좋았습니다. 그리고 고궁 자체가 넓으니까 사람들이 엄청 붐비는 느낌도 들지 않아서 걸어가면사 편하게 구경들 할수 있을거라고 생각합니다.
4899 KatyK 4.7 5.0 경복궁 야간관람 너무너무 좋아요! 요즘 잘 보고 있는 선을 넘는 녀석들 흥선대원군편 보고 관람하니 뭔가 느낌도 색달랐던 것 같고 킹덤3도 기다려집 니다. 다만 네이버 예약을 통해 이미 QR코드를 발급받았음에도 불구하고 매표소 좌측에 키오스크에서 핸드폰번호를 입력하고 지류 입장권을 뽑아서 입장을 해야되는 점이 번거로웠습니다. 줄이 매표소에서부터 고궁박물관이 있는 곳까지 정말 길었어요. 물론 금방금방 줄어들긴 했지만 이런 ...펼쳐보기
4900 mug14 5.0 5.0 편하게 이용해서 좋았습니다.
4901 forest 4.5 5.0 재밌고 너무 좋았어요
4902 tbr**** 1.5 1.5 예매 시 개인 QR이 나오는데도 그걸 활용하지 않고, 줄을 서서 부스에서 전화번호를 입력해서 종이 티켓을 출력하게 만드는 비효율이 이해가 가지 않음. 발열체크나 손세정제 등 보건 관련 안내도 별도로 없어서 쓰는 사람, 안 쓰는 사람 따로 있음. 현장구매 줄은 텅텅 비었는데 역설적으로 미리 예 매한 사람들만 줄을 한참 서게 만들었고, 터치 스크린이라 찝찝한데도 손세정제 하나 근처

5022 hye won 3.5 3.5 한번 가볍게 산책하기 좋아요
5023 까꿍 4.9 4.5 아이들이랑 같이 갔는데 너무 멋진 경복궁 야경이였습니다. 다만 젊은이들이 한복입고 소란스러워서 분위기는 별로였어요 성숙한 관람문화가 생기길 바랍니다. 야경이랑 날씨랑 너무 좋았어요
5024 양수정 5.0 5.0 정말 좋았어요!!
5025 지형맘 5.0 5.0 너무 아름답고 웅장함에 관람하며 우와~대단하다! 를 계속했어요.다음에는 꼭 한복입고 가보게요!
5026 MapleS 4.0 5.0 야간의 궁 관광은 너무나 운치있고 위엄있네요. 즐거웠어요!
5027 RiAn 4.6 5.0 다른 계절에 또 가고 싶네요
5028 케이디노 4.8 5.0 꼼꼼하게 관리해서 오래오래 보게 해주세여 너무 좋은 곳입니다
5029 이지룩 3.7 5.0 야경도 멋지고 참 좋은 경험이었어요
5030 시샨 4.7 5.0 모두 다 출사갈 분위기! 너무 예뻐요.
5031 사무용 4.8 5.0 경복궁 최고 날씨까지 바처주네요 ㅎㅅ
5032 lyjin0405 4.6 5.0 코로나영향으로 사람도 많지않고 한적해서 좋았어요 다만 야간개장이지만 너무 어두웠어요ㅠ
5033 김예진 5.0 5.0 날씨도 좋고 하늘도 맑아서 너무 좋았습니다 분위기도 좋고 궁궐 볼 때마다 계속 감탄해요
5034 효원 5.0 5.0 좋아요😍
5035 원필름 4.0 4.0 좋아요
5036 freeout02 4.9 5.0 ㄱ
5037 cpz**** 5.0 5.0 정말 좋아요
5038 유예리미 4.1 5.0 미리 예매하고 가니까 편리하고 좋아요! 야간관람 너무 이뻐요
5039 허둥 4.6 5.0 가성비 좋습니다 한시간정도 산책겸 다니기 좋습니다
5040 수안 4.9 5.0 낮에 보는 경복궁과 밤에 보는 경복궁은 또 다르네요. 정말 너무 너무 예뻤습니다.
5041 vwj**** 4.5 4.5 짱이뻐~~~~~~~
5042 hig**** 5.0 5.0 예약제라 그런지 사람도 많이 없어요 ㅎㅎ..그냥 강추입니다!
5043 이지우 5.0 5.0 경복궁

5182 이승원 4.7 5.0 너무너무 좋았어요 ㅎㅎㅎ
5183 냐하하핫 4.8 5.0 날씨도 좋고 다 좋았어요
5184 coco0301 3.2 5.0 감사합니다. ^^
5185 lkb**** 4.8 5.0 날씨도 시원하고 좋아요
5186 tka**** 5.0 5.0 실내 데이트 하기에 좋은거같아요. 조용하고 예쁜 전시회 보고싶으신 분들께 추천해드리고싶네요.
5187 달걀로바위치기 4.6 4.5 너무 이뻐요 조명도 잔잔하고
5188 eld**** 5.0 5.0 좋았어요 다만 불빛이 조금어두워서 사진찍기 힘들었어요
5189 할로워즈 4.5 4.5 분위기 좋고, 제가 원하는 것도 다 보고 와서 좋기는 합니다. 밤이라 그런가 뚜렷하게 보이지는 않았지만, 밤에서만 보이는 사물의 형상이나 사물의 모 습이 또 색다르게 느껴지기도 했구요. 경복궁 바로 밖과 안의 분위기 차이가 굉장이 심하게 차이가 났던 것 같습니다. 도시 속에서 고요함의 느낌이라 고 해야되나.. 뭐 굳이 꼽는 단점 정도라면 본인이 야맹증이 있다면 다닐때 조금씩 조심해야될 것 같네요. (돌 부리 걸려서 자빠질뻔 했거든요 ㅜㅡ..)펼쳐보기
5190 yrw**** 4.6 4.5 경복궁 야간 관람 너무 좋았어요~ 코로나로 한달간 어디 가지도 않고 집콕이었는데 밤에 시원한 공기도 좋고 사람도 많지 않아서 괜찮았네요
5191 초코홀릭 5.0 5.0 너무 멋있고 좋았어요
5192 425rose 4.5 4.5 날씨도 선선해서 걷기좋았어요!! 한시간정도 잡으면 충분히 다 볼수있어요
5193 깍미 4.5 5.0 감사해욥 멋진 밤이었어요..
5194 메주 5.0 5.0 3번째 방문인데 코로나 여파로 사람이 없어서 편하게 관람했어요. 밤에 보는 경복궁 정말 예뻐요 ~
5195 lgh**** 5.0 5.0 경복궁이 좋은건지 제가 좋아하는사람이랑 가서 좋은건지 평생 잊지못할 하루였어요
5196 sarchu 4.1 3.5 날도좋고 시원하고 낮과는 다른멋에 만족
5197 유림 5.0 5.0 너무 이쁘고ㅠ좋았어요!
5198 2umi 4

In [61]:
user_normal_data = pd.DataFrame(columns= ["UserID", "User_Avg","User_Spot_Avg","Review_text"])
user_normal_data['UserID']= user_id
user_normal_data['User_Avg']= user_avg
user_normal_data['User_Spot_Avg']=user_Spot_avg
user_normal_data['Review_text']=user_text
user_normal_data

,UserID,User_Avg,User_Spot_Avg,Review_text
0,라떼,4.9,5.0,좋네요~~
1,히카루,4.9,5.0,텍스트 후기 없음
2,장다르메,4.5,5.0,텍스트 후기 없음
3,kj9948,5.0,5.0,감사합니다
4,starry night,4.9,5.0,엄청넓어요
...,...,...,...,...
5330,구름칠111,5.0,5.0,J
5331,용의군사,5.0,5.0,사진 찍으러 자주 가는 곳. 향원정 빨리 보고 싶어요
5332,유츄시,4.4,5.0,갔다온지는 한참됐지만 ㅎㅎ 정말 예뻤습니다!
5333,도롤롱,4.2,5.0,텍스트 후기 없음


In [74]:
# 일반(텍스트 포함)리뷰 데이터 총 본 생성
user_normal_data.to_csv("경복궁_일반(텍스트 포함)리뷰 총 데이터.csv", encoding='utf-8-sig',index=False)

In [79]:
#### 이런 데이터가 중복이 많음,,,ㅎ...
total_normal_set = set([tuple(k) for k in total_normal])
print('중복 제거 전 일반(텍스트 포함)리뷰 수는', len(total_normal) )
print('중복 제거 후 일반(텍스트 포함)리뷰 수는', len(total_normal_set) )

중복 제거 전 일반(텍스트 포함)리뷰 수는 5335
중복 제거 후 일반(텍스트 포함)리뷰 수는 5328


#### 별점(텍스트 없는)리뷰 : userID, user의 평균 평점, user의 해당 명소 평점, 텍스트 후기

In [109]:
#Step 1. 크롬 웹브라우저 실행
path = "chromedriver.exe"  # 맥 기준  # 윈도우는 "chromedriver.exe"
driver = webdriver.Chrome(path)

# 사이트 주소는 네이버 마이 플레이스
driver.get('https://m.naver.com/')
time.sleep(2)

#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_xpath("""//*[@id="MM_SEARCH_FAKE"]""").click()
element_search = driver.find_element_by_xpath("""//*[@id="query"]""")
element_search.send_keys(spot)  # query_txt는 위에서 입력한 키워드
element_search.submit()
time.sleep(1)

#Step 3. '방문자 리뷰 더보기' 클릭
driver.find_element_by_xpath("""//*[@id="place-main-section-root"]/div/div[3]/div[2]/a""").click()

def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 99999999)")
    time.sleep(1)

scroll_down(driver) # 스크롤 다운

#Step 4. '별점리뷰'의 '펼처보기'클릭
driver.find_element_by_css_selector('a.eJXKm').click()
scroll_down(driver) # 스크롤 다운

In [111]:
# 총 반복 횟수
push2 = star//10
print('별점(텍스트 없는)리뷰 페이지 수 :', push2)

# '더보기' 누르기
for i in tqdm_notebook(range (push2)):
    scroll_down(driver) # 스크롤 다운
    # [더보기] 클릭
    driver.find_element_by_xpath("""//*[@id="app-root"]/div/div[2]/div[5]/div[4]/div[5]/div[2]""").click()
     
# css_selecotor로 찾은 별점(텍스트 없는)리뷰 페이지 수
review_overalls = driver.find_elements_by_css_selector('.HJ4ys')
print("css_selecotor로 찾은 별점(텍스트 없는)리뷰 페이지 수",len(review_overalls))

별점(텍스트 없는)리뷰 페이지 수 : 85



css_selecotor로 찾은 별점(텍스트 없는)리뷰 페이지 수 850


In [118]:
# 값들 가져오기
total_star = []
user_id_star= []
user_avg_star = []
user_Spot_avg_star =[]
user_text_star = []

for i in tqdm_notebook(range (star)):
    review_overalls = driver.find_elements_by_css_selector('.HJ4ys')
    information = review_overalls[i].text.split('\n')
    total_star.append(information)
    
    # user_id_star
    user_name_star = information[0]
    user_id_star.append(user_name_star)

    # user_avg_star
    user_avg_value_star = float(information[3])
    user_avg_star.append(user_avg_value_star)

    # user_Spot_avg_star
    user_Spot_avg_value_star = float(information[4])
    user_Spot_avg_star.append(user_Spot_avg_value_star)
    
    print(i+1, user_name_star, user_avg_value_star, user_Spot_avg_value_star)

#print(user_id_star)
#print(user_avg_star)
#print(user_Spot_avg_star)

1 쉐이크 4.5 5.0
2 엔돌핀 4.0 4.0
3 shsnu 4.5 4.5
4 loi**** 4.0 5.0
5 Melody 2.8 3.0
6 say 4.9 5.0
7 헬로욜 4.7 4.5
8 임마누엘 4.5 4.5
9 wjd**** 4.9 5.0
10 밝고긍정적으로 4.4 5.0
11 밝고긍정적으로 4.4 5.0
12 rla**** 4.8 5.0
13 whi**** 3.8 3.5
14 자두언냐 4.8 5.0
15 ㅇㅎㅈ 4.1 5.0
16 하트heart 4.9 4.5
17 kim**** 4.2 4.0
18 smh8834 4.9 5.0
19 망고 5.0 5.0
20 루루 4.6 5.0
21 직업상담사를위하여 3.8 4.0
22 heo**** 5.0 5.0
23 cau**** 4.3 4.5
24 CTIWIG 4.9 5.0
25 sportlim 4.7 5.0
26 Greg 4.9 5.0
27 gkp**** 4.0 4.0
28 쭈뚜쭈아 5.0 5.0
29 가시꽂 4.6 4.5
30 nij**** 5.0 5.0
31 jini080801 4.1 4.5
32 창문 4.0 4.5
33 chichi 4.7 4.0
34 dlr**** 4.1 4.0
35 안뇽 4.7 5.0
36 jio**** 4.4 5.0
37 유진향 4.0 4.0
38 행복맘마 3.0 3.5
39 주희 5.0 5.0
40 ck-**** 3.0 3.0
41 gut 4.9 5.0
42 yhw 4.6 5.0
43 tan**** 3.8 4.0
44 qja**** 4.9 5.0
45 이현수 4.2 5.0
46 lem**** 4.0 4.5
47 오아시스 4.3 5.0
48 joo**** 3.8 5.0
49 승호짱짱 4.9 5.0
50 민락공주 4.9 5.0
51 미애그라 4.4 4.5
52 서지영 5.0 5.0
53 울산 5.0 5.0
54 ㅎㅎ 5.0 5.0
55 육식꼬마 2.7 4.5
56 서율아빠 5.0 5.0
57 ys12jj 4.6 5.0
58 ANNA 4.0 3.0
59 밍밍이 3.9 4.0
60 okk*

462 ths**** 4.0 4.0
463 SIS 3.9 4.0
464 민스리맘 4.0 4.0
465 qch**** 4.8 4.5
466 호옹이 5.0 5.0
467 정미영 4.7 4.5
468 이음 4.4 5.0
469 max**** 3.4 3.5
470 Lion 4.7 5.0
471 sel**** 4.7 4.5
472 hyu**** 4.7 5.0
473 피넛 4.9 5.0
474 ajang75 3.6 5.0
475 강태인 4.8 5.0
476 나부토이 4.5 5.0
477 mih**** 4.2 4.0
478 jih**** 5.0 5.0
479 aa5**** 4.0 4.0
480 뽈타치 3.9 4.0
481 핸니미 4.6 5.0
482 효진구리 4.2 4.5
483 Killerwhale 4.4 4.5
484 cny**** 4.1 4.0
485 영주 5.0 5.0
486 찜니 4.9 4.5
487 lov**** 4.9 4.5
488 Kkambu 3.8 4.5
489 그리피스 4.8 5.0
490 루니짱 3.8 3.0
491 zest 4.2 4.5
492 aut**** 4.3 3.0
493 air**** 3.2 4.0
494 hex**** 4.4 4.5
495 정미현 3.0 3.0
496 sky**** 4.8 4.5
497 레몬트리 4.0 4.0
498 로라애슐리맘 3.7 3.5
499 inh**** 4.5 4.0
500 son**** 4.2 4.0
501 dow**** 5.0 5.0
502 pop**** 4.8 5.0
503 seu**** 4.4 4.5
504 minime3 5.0 5.0
505 jon**** 4.7 4.0
506 hahaha123 4.2 4.0
507 dre**** 3.3 3.5
508 wld**** 4.9 4.5
509 dre**** 3.3 4.0
510 wls**** 4.0 5.0
511 박성아 4.6 5.0
512 멍뭉 5.0 5.0
513 jin**** 4.5 5.0
514 Joel 5.0 5.0
515 2tk**** 4.7 5.0
5

In [120]:
user_star_data = pd.DataFrame({'UserID':user_id_star, 'User_Avg':user_avg_star,'User_Spot_Avg':user_Spot_avg_star})
user_star_data.to_csv("경복궁_별점(텍스트 없는)리뷰.csv", encoding="utf-8-sig", index=False)

### 블로그 리뷰 크롤링

추가로 할까 했으나,,,, 필요가 없을 듯,텍스트 분석을 할 것은 아니기에,,,, 위와 같은 방식으로 제목 가져오면 됨...ㅎ